### [Reference Stackoverflow](https://stackoverflow.com/questions/51030782/why-do-we-pack-the-sequences-in-pytorch)

In [2]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence
from torch.nn.utils.rnn import pad_packed_sequence

In [3]:
import torch 

seq_batch = [torch.tensor([[1, 1],
                           [2, 2],
                           [3, 3],
                           [4, 4],
                           [5, 5]]),
             torch.tensor([[10, 10],
                           [20, 20]])]

seq_lens = [5, 2]

In [4]:
# pad the seq_batch
padded_seq_batch = torch.nn.utils.rnn.pad_sequence(seq_batch, batch_first=True)
"""
>>>padded_seq_batch
tensor([[[ 1,  1],
         [ 2,  2],
         [ 3,  3],
         [ 4,  4],
         [ 5,  5]],

        [[10, 10],
         [20, 20],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0]]])
"""

'\n>>>padded_seq_batch\ntensor([[[ 1,  1],\n         [ 2,  2],\n         [ 3,  3],\n         [ 4,  4],\n         [ 5,  5]],\n\n        [[10, 10],\n         [20, 20],\n         [ 0,  0],\n         [ 0,  0],\n         [ 0,  0]]])\n'

In [15]:
padded_seq_batch

tensor([[[ 1,  1],
         [ 2,  2],
         [ 3,  3],
         [ 4,  4],
         [ 5,  5]],

        [[10, 10],
         [20, 20],
         [ 0,  0],
         [ 0,  0],
         [ 0,  0]]])

In [16]:
# pack the padded_seq_batch
packed_seq_batch = torch.nn.utils.rnn.pack_padded_sequence(padded_seq_batch, lengths=seq_lens, batch_first=True)
"""
>>> packed_seq_batch
PackedSequence(
   data=tensor([[ 1,  1],
                [10, 10],
                [ 2,  2],
                [20, 20],
                [ 3,  3],
                [ 4,  4],
                [ 5,  5]]), 
   batch_sizes=tensor([2, 2, 1, 1, 1]))
"""
packed_seq_batch

PackedSequence(data=tensor([[ 1,  1],
        [10, 10],
        [ 2,  2],
        [20, 20],
        [ 3,  3],
        [ 4,  4],
        [ 5,  5]]), batch_sizes=tensor([2, 2, 1, 1, 1]), sorted_indices=None, unsorted_indices=None)

In [8]:
# input size : input vector size of each time stamp
# hidden size : hidden vector size(output)
lstm = nn.LSTM(input_size=2, hidden_size=3, batch_first=True)
output, (hn, cn) = lstm(packed_seq_batch.float())

In [9]:
output

PackedSequence(data=tensor([[9.6369e-03, 7.6992e-02, 1.3758e-02],
        [1.0080e-01, 3.6183e-03, 1.9047e-05],
        [9.7091e-02, 1.1201e-01, 4.0262e-02],
        [2.7153e-02, 3.2999e-03, 2.0086e-05],
        [1.4345e-01, 1.2920e-01, 5.7135e-02],
        [1.5237e-01, 1.3640e-01, 6.6257e-02],
        [1.4731e-01, 1.3819e-01, 7.1561e-02]], grad_fn=<CatBackward0>), batch_sizes=tensor([2, 2, 1, 1, 1]), sorted_indices=None, unsorted_indices=None)

In [10]:
hn

tensor([[[1.4731e-01, 1.3819e-01, 7.1561e-02],
         [2.7153e-02, 3.2999e-03, 2.0086e-05]]], grad_fn=<StackBackward0>)

In [11]:
cn

tensor([[[5.0977e-01, 6.1316e-01, 9.3382e-02],
         [9.1511e-01, 1.6663e-02, 2.0128e-05]]], grad_fn=<StackBackward0>)

In [12]:
padded_output, output_lens = torch.nn.utils.rnn.pad_packed_sequence(output, batch_first=True)

In [13]:
padded_output

tensor([[[9.6369e-03, 7.6992e-02, 1.3758e-02],
         [9.7091e-02, 1.1201e-01, 4.0262e-02],
         [1.4345e-01, 1.2920e-01, 5.7135e-02],
         [1.5237e-01, 1.3640e-01, 6.6257e-02],
         [1.4731e-01, 1.3819e-01, 7.1561e-02]],

        [[1.0080e-01, 3.6183e-03, 1.9047e-05],
         [2.7153e-02, 3.2999e-03, 2.0086e-05],
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00]]], grad_fn=<TransposeBackward0>)

In [14]:
output_lens

tensor([5, 2])

In [81]:
cumulative_probs = [torch.FloatTensor([.0] + [-float('inf')] * (5 - 1))]
a = cumulative_probs + cumulative_probs
a[-1]

tensor([0., -inf, -inf, -inf, -inf])

In [49]:
masks = [torch.BoolTensor(5).zero_()]
masks

[tensor([False, False, False, False, False])]

In [79]:
torch

TypeError: eq() received an invalid combination of arguments - got (str, int), but expected one of:
 * (Tensor input, Tensor other, *, Tensor out)
 * (Tensor input, Number other, *, Tensor out)


In [50]:
cumulative_probs[-1].masked_fill_(masks[-1], -float('inf'))

tensor([0., -inf, -inf, -inf, -inf])

In [54]:
torch.FloatTensor([5, -float('inf'), 8, 6, -float("inf")]).view(-1, 1, 1).expand(5, 1, 100)

tensor([[[5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
          5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
          5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
          5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
          5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
          5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.]],

        [[-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -

In [57]:
torch.FloatTensor([5, -float('inf'), 8, 6, -float("inf")]).view(-1, 1, 1).expand(5, 1, 100)

tensor([[[5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
          5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
          5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
          5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
          5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
          5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.]],

        [[-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -

In [73]:
torch.FloatTensor([5, -float('inf'), 8, 6, -float("inf")]).view(-1, 1, 1).expand(5, 1, 100)

tensor([[[5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
          5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
          5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
          5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
          5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
          5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.]],

        [[-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -

In [84]:
probs = []
probs += [0.3]
probs += [0.7]

In [85]:
probs

[0.3, 0.7]

In [100]:
torch.tensor(np.array([[1, 2, 3], [4, 5, 6]]))*2

tensor([[ 2,  4,  6],
        [ 8, 10, 12]])

In [105]:
[[1, 2, 3], [4, 5, 6]]

[[1, 2, 3], [4, 5, 6]]

In [107]:
[[1, 2, 3], [4, 5, 6]] * 3

[[1, 2, 3], [4, 5, 6], [1, 2, 3], [4, 5, 6], [1, 2, 3], [4, 5, 6]]

In [123]:
hi = []+[torch.rand(2, )]

torch.cat(hi,  dim=0).shape

torch.Size([2])

In [125]:
[torch.rand(2, )]

[tensor([0.8672, 0.9692])]

In [116]:
torch.cat([torch.rand(2, 1, 3)]*3, dim=1).shape

torch.Size([2, 3, 3])

In [129]:
fab_hidden = []+[torch.rand(2, 5, 3)]
torch.cat(fab_hidden, dim=1).shape

torch.Size([2, 5, 3])

In [131]:
abc=[]
h_src = torch.rand(2, 10, 3)
abc += [h_src[1, :, :]] * 3

In [137]:
torch.stack(abc).shape

torch.Size([3, 10, 3])

In [136]:
abc

[tensor([[0.4736, 0.2630, 0.7601],
         [0.6965, 0.7868, 0.4692],
         [0.6470, 0.3949, 0.8984],
         [0.4147, 0.1759, 0.7857],
         [0.0977, 0.6804, 0.9706],
         [0.1931, 0.6092, 0.2824],
         [0.6803, 0.4303, 0.0380],
         [0.1541, 0.2444, 0.5001],
         [0.8880, 0.9470, 0.7729],
         [0.2098, 0.0514, 0.0766]]),
 tensor([[0.4736, 0.2630, 0.7601],
         [0.6965, 0.7868, 0.4692],
         [0.6470, 0.3949, 0.8984],
         [0.4147, 0.1759, 0.7857],
         [0.0977, 0.6804, 0.9706],
         [0.1931, 0.6092, 0.2824],
         [0.6803, 0.4303, 0.0380],
         [0.1541, 0.2444, 0.5001],
         [0.8880, 0.9470, 0.7729],
         [0.2098, 0.0514, 0.0766]]),
 tensor([[0.4736, 0.2630, 0.7601],
         [0.6965, 0.7868, 0.4692],
         [0.6470, 0.3949, 0.8984],
         [0.4147, 0.1759, 0.7857],
         [0.0977, 0.6804, 0.9706],
         [0.1931, 0.6092, 0.2824],
         [0.6803, 0.4303, 0.0380],
         [0.1541, 0.2444, 0.5001],
         [0.8880

In [159]:
hidden_size = 20
max_length = 10

In [160]:
enc = torch.FloatTensor(max_length, hidden_size).zero_()

In [161]:
pos = torch.arange(0, max_length).unsqueeze(-1).float()
dim = torch.arange(0, hidden_size // 2).unsqueeze(0).float()

In [162]:
pos

tensor([[0.],
        [1.],
        [2.],
        [3.],
        [4.],
        [5.],
        [6.],
        [7.],
        [8.],
        [9.]])

In [163]:
dim

tensor([[0., 1., 2., 3., 4., 5., 6., 7., 8., 9.]])

In [169]:
torch.sin(pos / 1e+4**dim.div(float(hidden_size)))

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.8415,  0.5899,  0.3877,  0.2486,  0.1578,  0.0998,  0.0631,  0.0398,
          0.0251,  0.0158],
        [ 0.9093,  0.9527,  0.7147,  0.4815,  0.3117,  0.1987,  0.1259,  0.0795,
          0.0502,  0.0317],
        [ 0.1411,  0.9486,  0.9300,  0.6842,  0.4578,  0.2955,  0.1882,  0.1191,
          0.0753,  0.0475],
        [-0.7568,  0.5792,  0.9998,  0.8440,  0.5923,  0.3894,  0.2497,  0.1586,
          0.1003,  0.0634],
        [-0.9589, -0.0132,  0.9132,  0.9508,  0.7121,  0.4794,  0.3103,  0.1977,
          0.1253,  0.0792],
        [-0.2794, -0.6005,  0.6838,  0.9980,  0.8140,  0.5646,  0.3696,  0.2366,
          0.1501,  0.0950],
        [ 0.6570, -0.9566,  0.3474,  0.9825,  0.8954,  0.6442,  0.4274,  0.2751,
          0.1749,  0.1107],
        [ 0.9894, -0.9443, -0.0433,  0.9053,  0.9545,  0.7174,  0.4836,  0.3131,
          0.1996,  0.1265],
        [ 0.4121, -

In [166]:
enc[:, 0::2] = torch.sin(pos / 1e+4**dim.div(float(hidden_size)))
enc[:, 1::2] = torch.cos(pos / 1e+4**dim.div(float(hidden_size)))

In [150]:
enc

tensor([[ 0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  1.0000,
          0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  1.0000,
          0.0000,  1.0000,  0.0000,  1.0000],
        [ 0.8415,  0.5403,  0.5899,  0.8075,  0.3877,  0.9218,  0.2486,  0.9686,
          0.1578,  0.9875,  0.0998,  0.9950,  0.0631,  0.9980,  0.0398,  0.9992,
          0.0251,  0.9997,  0.0158,  0.9999],
        [ 0.9093, -0.4161,  0.9527,  0.3040,  0.7147,  0.6994,  0.4815,  0.8764,
          0.3117,  0.9502,  0.1987,  0.9801,  0.1259,  0.9920,  0.0795,  0.9968,
          0.0502,  0.9987,  0.0317,  0.9995],
        [ 0.1411, -0.9900,  0.9486, -0.3165,  0.9300,  0.3676,  0.6842,  0.7293,
          0.4578,  0.8891,  0.2955,  0.9553,  0.1882,  0.9821,  0.1191,  0.9929,
          0.0753,  0.9972,  0.0475,  0.9989],
        [-0.7568, -0.6536,  0.5792, -0.8152,  0.9998, -0.0216,  0.8440,  0.5363,
          0.5923,  0.8057,  0.3894,  0.9211,  0.2497,  0.9683,  0.1586,  0.9873,
      

In [179]:
enc[:3, :2] 

tensor([[ 0.0000,  1.0000],
        [ 0.8415,  0.5403],
        [ 0.9093, -0.4161]])

In [189]:
matrix = torch.Tensor([[0, 0, 0, 1], [0, 1, 1, 1]])
matrix

tensor([[0., 0., 0., 1.],
        [0., 1., 1., 1.]])

In [190]:
matrix.shape

torch.Size([2, 4])

In [191]:
matrix.unsqueeze(1).expand(2, 4, 4)

tensor([[[0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.]],

        [[0., 1., 1., 1.],
         [0., 1., 1., 1.],
         [0., 1., 1., 1.],
         [0., 1., 1., 1.]]])

In [192]:
matrix.unsqueeze(1).expand(2, 4, 4).shape

torch.Size([2, 4, 4])

In [193]:
matrix.unsqueeze(1).expand(2, 3, 4)

tensor([[[0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.]],

        [[0., 1., 1., 1.],
         [0., 1., 1., 1.],
         [0., 1., 1., 1.]]])

In [195]:
# upper triangle

In [194]:
torch.triu(matrix.new_ones(
                (3, 3)), diagonal=1).bool()

tensor([[False,  True,  True],
        [False, False,  True],
        [False, False, False]])

In [196]:
dict_sample = {'a': [1, 2, 3]}

In [197]:
dict_sample['a']+=[2]

In [198]:
dict_sample

{'a': [1, 2, 3, 2]}

In [200]:
[[1, 2, 3]]*2

[[1, 2, 3], [1, 2, 3]]

In [216]:
torch.Tensor(1, 3)

tensor([[0.0000e+00, 4.6566e-10, 0.0000e+00]])

In [227]:
torch.Tensor(3, 1)

tensor([[0.0000e+00],
        [4.6566e-10],
        [2.3370e-19]])

In [228]:
fab_input = []
fab_input += [torch.Tensor(3, 1)]
fab_input += [torch.Tensor(3, 1)]
torch.cat(fab_input, dim=0)

tensor([[9.1477e-41],
        [0.0000e+00],
        [1.6816e-44],
        [0.0000e+00],
        [0.0000e+00],
        [3.5993e-04]])

In [218]:
torch.cat(fab_input, dim=0)

tensor([0., 0., 0., 0., 0., 0.])